In [3]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict

# -------------------------------------------------------------
# Configuration
# -------------------------------------------------------------
ANNOTATION_JSON = "../annotations/annotations.json"  # Adjust the path to your annotations file
OUTPUT_DIR = "../results"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# -------------------------------------------------------------
# Load Annotations
# -------------------------------------------------------------
print("Loading annotations from:", ANNOTATION_JSON)

with open(ANNOTATION_JSON, 'r') as f:
    annotations_data = json.load(f)

# -------------------------------------------------------------
# Helper: Extract keypoints per image
# -------------------------------------------------------------
images_data = []
label_count = defaultdict(int)
invalid_annotations = []

# List to store number of keypoints per image
annotations_per_image = []

for image in annotations_data['images']:
    img_name = image['file_name']
    width = image['width']
    height = image['height']
    
    annotations = [ann for ann in annotations_data['annotations'] if ann['image_id'] == image['id']]
    
    image_points = []
    for ann in annotations:
        keypoints = ann['keypoints']
        valid_points = []
        
        # Check and collect valid points (no occlusion and within bounds)
        for i in range(0, len(keypoints), 3):
            x, y, visibility = keypoints[i], keypoints[i + 1], keypoints[i + 2]
            if 0 <= x <= width and 0 <= y <= height:
                valid_points.append((x, y))
                label_count['keypoints'] += 1
            else:
                invalid_annotations.append((img_name, f"Point {i//3 + 1} out of bounds"))
        
        # Store the points
        image_points.append({"image": img_name, "points": valid_points})
    
    # Store the count of annotations for the current image
    annotations_per_image.append({"image": img_name, "num_keypoints": len(image_points[0]['points'])})

images_data.extend(image_points)

# -------------------------------------------------------------
# Basic Statistics
# -------------------------------------------------------------
total_images = len(annotations_data['images'])
total_annotations = sum(d["num_keypoints"] for d in annotations_per_image)
total_labels = len(label_count)

print(f"Total Images: {total_images}")
print(f"Total Annotations (points): {total_annotations}")
print(f"Total Labels: {total_labels}")

# -------------------------------------------------------------
# Save Statistics
# -------------------------------------------------------------
stats_path = os.path.join(OUTPUT_DIR, "annotations_stats.txt")
with open(stats_path, "w") as f:
    f.write("=== Body Landmarks Annotation Statistics ===\n\n")
    f.write(f"Total Images: {total_images}\n")
    f.write(f"Total Annotations (points): {total_annotations}\n")
    f.write(f"Total Labels: {total_labels}\n\n")
    
    f.write("--- Annotations per Image ---\n")
    for img_data in annotations_per_image:
        f.write(f"{img_data['image']}: {img_data['num_keypoints']} keypoints\n")
    
    f.write("\n--- Invalid Annotations (out-of-bound points) ---\n")
    for img, label in invalid_annotations:
        f.write(f"{img} -> {label}\n")

print("Saved statistics to:", stats_path)

# -------------------------------------------------------------
# Bar Plot for Annotations per Image
# -------------------------------------------------------------
# Create a bar plot showing the number of keypoints per image
image_names = [img_data['image'] for img_data in annotations_per_image]
keypoint_counts = [img_data['num_keypoints'] for img_data in annotations_per_image]

plt.figure(figsize=(12, 6))
plt.bar(image_names, keypoint_counts, color='skyblue')
plt.xlabel("Image")
plt.ylabel("Number of Keypoints")
plt.title("Number of Keypoints Annotated per Image")
plt.xticks(rotation=45, ha="right")  # Rotate the x-axis labels for better readability
plt.tight_layout()

# Save the plot
keypoint_bar_plot_path = os.path.join(OUTPUT_DIR, "keypoints_per_image_bar_plot.png")
plt.savefig(keypoint_bar_plot_path)
plt.close()

print("Saved bar plot for keypoints per image to:", keypoint_bar_plot_path)

# -------------------------------------------------------------
# Evaluation Summary
# -------------------------------------------------------------
all_points_count = [d["num_keypoints"] for d in annotations_per_image]

eval_path = os.path.join(OUTPUT_DIR, "eval_summary.txt")
with open(eval_path, "w") as f:
    f.write("=== Evaluation Summary (Point Quality Checks) ===\n\n")
    f.write(f"Invalid Annotations (out-of-bound): {len(invalid_annotations)}\n")
    f.write(f"Mean points per image: {sum(all_points_count) / len(all_points_count):.2f}\n")
    f.write(f"Median points per image: {sorted(all_points_count)[len(all_points_count)//2]:.2f}\n")
    f.write(f"Max points per image: {max(all_points_count):.2f}\n")

print("Saved evaluation summary to:", eval_path)

# -------------------------------------------------------------
# Markdown Report
# -------------------------------------------------------------
report_path = os.path.join(OUTPUT_DIR, "report.md")
with open(report_path, "w") as f:
    f.write("# Body Landmarks Annotation Report\n")
    f.write("Generated automatically from annotations.\n\n")
    
    f.write("## Dataset Summary\n")
    f.write(f"- **Images:** {total_images}\n")
    f.write(f"- **Annotations (Points):** {total_annotations}\n")
    f.write(f"- **Total Labels:** {total_labels}\n\n")
    
    f.write("## Annotation Quality Checks\n")
    f.write(f"- Invalid Annotations (out-of-bound points): **{len(invalid_annotations)}**\n\n")
    
    f.write("## Keypoints Distribution per Image\n")
    f.write("![](keypoints_per_image_bar_plot.png)\n\n")
    
    f.write("## Notes\n")
    f.write("- Analyzed annotations for **body landmarks**.\n")
    f.write("- No skeleton visualization performed.\n")

print("Saved report to:", report_path)

print("\n=== ALL DONE! ===")
print("Results saved to:", OUTPUT_DIR)


Loading annotations from: ../annotations/annotations.json
Total Images: 12
Total Annotations (points): 204
Total Labels: 1
Saved statistics to: ../results/annotations_stats.txt
Saved bar plot for keypoints per image to: ../results/keypoints_per_image_bar_plot.png
Saved evaluation summary to: ../results/eval_summary.txt
Saved report to: ../results/report.md

=== ALL DONE! ===
Results saved to: ../results
